In [1]:
# Read text files
from gurobipy import *
import numpy as np
import math
import matplotlib.pyplot

In [2]:
# Extract coordiantes and orgainze them
airport = ['a','b','c']
combination = ['ab','ac','ba','bc','ca','cb']
come_cargo_dic= {'ab': [100, 200, 100, 400, 300],
'ac':[50, 50, 50, 50, 50],
'ba':[25, 25, 25, 25, 25],
'bc':[25, 25, 25, 25, 25],
'ca':[40, 40, 40, 40, 40],
'cb':[400, 200, 300, 200, 400]}

### decision variable 

In [3]:
# Create decision variables x and store them in the arrary myVars

myModel = Model( "Project" )
E_var = {}
for d in range (1,6):
    E_var[d] = {}
    for i in range (1,7):
        E_var[d][combination[i-1]] = {}
        name_var = "E" + str(d) + combination[i-1]
        E_var[d][combination[i-1]] = myModel.addVar( vtype = GRB.CONTINUOUS, name =  name_var )

H_var = {}
for d in range (1,6):
    H_var[d] = {}
    for i in range (1,7):
        H_var[d][combination[i-1]] = {}
        name_var =  "H" + str(d) + combination[i-1]
        H_var[d][combination[i-1]] = myModel.addVar( vtype = GRB.CONTINUOUS, name = name_var )
        
F_var = {}
for d in range (1,6):
    F_var[d] = {}
    for i in range (1,7):
        F_var[d][combination[i-1]] = {}
        name_var =  "F" + str(d) + combination[i-1]
        F_var[d][combination[i-1]] = myModel.addVar( vtype = GRB.CONTINUOUS, name =  name_var )

I_var = {}
for d in range (1,6):
    I_var[d] = {}
    for i in range (1,4):
        I_var[d][airport[i-1]] = {}
        name_var = "I" + str(d) + airport[i-1]
        I_var[d][airport[i-1]] = myModel.addVar( vtype = GRB.CONTINUOUS, name = name_var )

myModel.update()
myVars = myModel.getVars()

Academic license - for non-commercial use only


In [4]:
print ('reposition variable:\n', [i.varName for i in myVars if i.varName[0] =='E'])
print ('cargo variable:\n', [i.varName for i in myVars if i.varName[0] =='F'])
print ('idle variable:\n', [i.varName for i in myVars if i.varName[0] =='I'])
print ('holding cargo:\n', [i.varName for i in myVars if i.varName[0] =='H'])

reposition variable:
 ['E1ab', 'E1ac', 'E1ba', 'E1bc', 'E1ca', 'E1cb', 'E2ab', 'E2ac', 'E2ba', 'E2bc', 'E2ca', 'E2cb', 'E3ab', 'E3ac', 'E3ba', 'E3bc', 'E3ca', 'E3cb', 'E4ab', 'E4ac', 'E4ba', 'E4bc', 'E4ca', 'E4cb', 'E5ab', 'E5ac', 'E5ba', 'E5bc', 'E5ca', 'E5cb']
cargo variable:
 ['F1ab', 'F1ac', 'F1ba', 'F1bc', 'F1ca', 'F1cb', 'F2ab', 'F2ac', 'F2ba', 'F2bc', 'F2ca', 'F2cb', 'F3ab', 'F3ac', 'F3ba', 'F3bc', 'F3ca', 'F3cb', 'F4ab', 'F4ac', 'F4ba', 'F4bc', 'F4ca', 'F4cb', 'F5ab', 'F5ac', 'F5ba', 'F5bc', 'F5ca', 'F5cb']
idle variable:
 ['I1a', 'I1b', 'I1c', 'I2a', 'I2b', 'I2c', 'I3a', 'I3b', 'I3c', 'I4a', 'I4b', 'I4c', 'I5a', 'I5b', 'I5c']
holding cargo:
 ['H1ab', 'H1ac', 'H1ba', 'H1bc', 'H1ca', 'H1cb', 'H2ab', 'H2ac', 'H2ba', 'H2bc', 'H2ca', 'H2cb', 'H3ab', 'H3ac', 'H3ba', 'H3bc', 'H3ca', 'H3cb', 'H4ab', 'H4ac', 'H4ba', 'H4bc', 'H4ca', 'H4cb', 'H5ab', 'H5ac', 'H5ba', 'H5bc', 'H5ca', 'H5cb']


### Objective function 

In [5]:
# Create a linear expression for the objective
objExpr = LinExpr()

for d in range (1,6):
    objExpr +=  (E_var[d]['ab'] + E_var[d]['ba']) *7 +\
    (E_var[d]['bc'] + E_var[d]['cb']) *6 +\
     (E_var[d]['ac'] + E_var[d]['ca']) *3   
for d in range (1,5):
    for c in combination:
        objExpr += H_var[d][c] * 10
for c in combination:
    objExpr += H_var[5][c] *30

myModel.setObjective(objExpr, GRB.MINIMIZE)
myModel.update()

###  constraints

In [6]:
# Create constraints
# Cargo part 
for c in combination:
    for d in range (1,5):
        constExpr = LinExpr()
        constExpr += H_var[d][c] + come_cargo_dic[c][d] - F_var[d+1][c]
        myModel.addConstr( lhs = constExpr , sense = GRB.EQUAL, \
                   rhs = H_var[d+1][c] , name = 'cargo_%s_%s'%(str(d),str(c)) )
    # special Friday
    constExpr = LinExpr()
    constExpr += H_var[5][c] + come_cargo_dic[c][0] - F_var[1][c]
    myModel.addConstr( lhs = constExpr , sense = GRB.EQUAL, \
                   rhs = H_var[1][c] , name = 'cargo_%s_%s'%(str(5),str(c)) )

In [7]:
#aircraft part 1
for d in range (1,5):
    for loc in ['a','b','c']:
    #left hand side 
        constExpr_l = LinExpr()
        constExpr_l += I_var[d][loc]
        # iterate all combination end with loc
        for com in [c for c in combination if c[1] == loc]:
            constExpr_l += E_var[d][com] + F_var[d][com]
        #right hand side 
        constExpr_r = LinExpr()
        constExpr_r += I_var[d+1][loc]
        # iterate all combination start with loc
        for com in [c for c in combination if c[0] == loc]:
            constExpr_r += E_var[d+1][com] + F_var[d+1][com]
        myModel.addConstr( lhs = constExpr_l , sense = GRB.EQUAL, \
                       rhs = constExpr_r , name = 'aircraft_1_%s_%s'%(str(d),str(loc)) )
# special Friday
d = 5
d_plus_1 = 1
for loc in ['a','b','c']:
    constExpr_l = LinExpr()
    constExpr_l += I_var[d][loc]
    # iterate all combination end with loc
    for com in [c for c in combination if c[1] == loc]:
        constExpr_l += E_var[d][com] + F_var[d][com]
    
    #right hand side 
    constExpr_r = LinExpr()
    constExpr_r += I_var[d_plus_1 ][loc]
    # iterate all combination start with loc
    for com in [c for c in combination if c[0] == loc]:
        constExpr_r += E_var[d_plus_1][com] + F_var[d_plus_1 ][com]
    myModel.addConstr( lhs = constExpr_l , sense = GRB.EQUAL, \
                   rhs = constExpr_r , name = 'aircraft_1_%s_%s'%(str(d),str(loc)) )


In [8]:
#aircraft part 2
# the index d here is for H_var, e.g F2ab = H1ab + come_cargo_dic[ab][1] (200)
for d in range (1,5):
    for c in combination:
        constExpr_r = LinExpr()
        constExpr_r += H_var[d][c] + come_cargo_dic[c][d]
        myModel.addConstr( lhs = F_var[d+1][c] , sense = GRB.LESS_EQUAL, \
                   rhs = constExpr_r , name = 'aircraft_2_%s_%s'%(str(d),str(c)) )
# special Friday
for c in combination:
    constExpr_r = LinExpr()
    constExpr_r += H_var[5][c] + come_cargo_dic[c][0]
    myModel.addConstr( lhs = F_var[1][c] , sense = GRB.LESS_EQUAL, \
               rhs = constExpr_r , name = 'aircraft_2_%s_%s'%(str(5),str(c)) )

In [9]:
#aircraft part 3
for d in range (1,6):
    constExpr_l = LinExpr()
    for item in list(E_var[d].values()) + list(F_var[d].values()) + list(I_var[d].values()):
        constExpr_l += item 
    myModel.addConstr(lhs =constExpr_l, sense = GRB.EQUAL, rhs =1200, \
                      name = 'aircraft_3_%s'%str(d) )

#aircraft part 4
for c in combination:
    constExpr_l = LinExpr()
    for d in range (1,6):
        constExpr_l += F_var[d][c]
    myModel.addConstr(lhs =constExpr_l, sense = GRB.EQUAL, rhs =np.sum(come_cargo_dic[c]), \
                      name = 'aircraft_4_%s_%s'%(str(c),str(d)) )
    
myModel.update()

In [10]:
# Solve
myModel.optimize()

Optimize a model with 86 rows, 105 columns and 405 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+03]
Presolve removed 11 rows and 2 columns
Presolve time: 0.02s
Presolved: 75 rows, 103 columns, 308 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   9.906250e+02   0.000000e+00      0s
      24    2.1725000e+04   0.000000e+00   0.000000e+00      0s

Solved in 24 iterations and 0.02 seconds
Optimal objective  2.172500000e+04


In [11]:
print( "\nOptimal Objective: " + str( myModel.ObjVal ) )
print( "\nOptimal Solution:" )
myVars = myModel.getVars()
for curVar in myVars:
    print ( curVar.varName + " " + str( curVar.x ) )
    


Optimal Objective: 21725.0

Optimal Solution:
E1ab 0.0
E1ac 0.0
E1ba 295.0
E1bc 165.0
E1ca 0.0
E1cb 0.0
E2ab 0.0
E2ac 0.0
E2ba 120.0
E2bc 430.0
E2ca 0.0
E2cb 0.0
E3ab 0.0
E3ac 0.0
E3ba 425.0
E3bc 0.0
E3ca 0.0
E3cb 0.0
E4ab 0.0
E4ac 0.0
E4ba 0.0
E4bc 365.0
E4ca 0.0
E4cb 0.0
E5ab 0.0
E5ac 0.0
E5ba 185.0
E5bc 365.0
E5ca 0.0
E5cb 0.0
H1ab 90.0
H1ac 0.0
H1ba 0.0
H1bc 0.0
H1ca 0.0
H1cb 0.0
H2ab 0.0
H2ac 0.0
H2ba 0.0
H2bc 0.0
H2ca 0.0
H2cb 0.0
H3ab 0.0
H3ac 0.0
H3ba 0.0
H3bc 0.0
H3ca 0.0
H3cb 0.0
H4ab 0.0
H4ac 0.0
H4ba 0.0
H4bc 0.0
H4ca 0.0
H4cb 0.0
H5ab 190.0
H5ac 0.0
H5ba 0.0
H5bc 0.0
H5ca 0.0
H5cb 0.0
F1ab 200.0
F1ac 50.0
F1ba 25.0
F1bc 25.0
F1ca 40.0
F1cb 400.0
F2ab 290.0
F2ac 50.0
F2ba 25.0
F2bc 25.0
F2ca 40.0
F2cb 200.0
F3ab 100.0
F3ac 50.0
F3ba 25.0
F3bc 25.0
F3ca 40.0
F3cb 300.0
F4ab 400.0
F4ac 50.0
F4ba 25.0
F4bc 25.0
F4ca 40.0
F4cb 200.0
F5ab 110.0
F5ac 50.0
F5ba 25.0
F5bc 25.0
F5ca 40.0
F5cb 400.0
I1a 0.0
I1b 0.0
I1c 0.0
I2a 20.0
I2b 0.0
I2c 0.0
I3a 55.0
I3b 15.0
I3c 165.0
I4a 95.

In [12]:
myModel.write( "project.lp" )